In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spellchecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import re
%matplotlib inline
%config IPCompleter.greedy=True

In [15]:
df = pd.read_csv("../train.csv")
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [24]:
spell = spellchecker.SpellChecker()
df["text"] = df.text.apply(lambda x: str(x).split())
df["Mispelled"] = df.text.apply(lambda x: spell.unknown(x))
df["checked"] = df.Mispelled.apply(lambda x: [spell.correction(z) for z  in x])
df.head()

KeyboardInterrupt: 

### Bag of Words Approach

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
df["text"]=df.text.apply(lambda x: re.sub("[^a-zA-Z' ]",' ', str(x)))
df=df[df["text"].notna()]
X_train = vectorizer.fit_transform(df["text"])

In [21]:
len(vectorizer.get_feature_names())

24304

In [23]:
cat_sentiment = {"positive":1,"neutral":0,"negative":-1}
df['cat_sentiment'] = df.sentiment.apply(lambda x: cat_sentiment[x])
df['cat_sentiment']

0        0
1       -1
2       -1
3       -1
4       -1
        ..
27476   -1
27477   -1
27478    1
27479    1
27480    0
Name: cat_sentiment, Length: 27481, dtype: int64

In [19]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
X_train_tfidf.shape
x_train, x_test, y_train, y_test = train_test_split(X_train_tfidf, df['cat_sentiment'], test_size=0.33, random_state=42)

(27481, 24304)

In [25]:
clf = MultinomialNB().fit(x_train, y_train)

In [37]:
predictions = clf.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print(f'accuracy: {accuracy}')
df_confusion = pd.crosstab(y_test, predictions, margins=True)
df_confusion

accuracy: 0.6198037269820267


col_0,-1,0,1,All
cat_sentiment,,,,
-1,1001,1418,142,2561
0,258,2959,492,3709
1,52,1086,1661,2799
All,1311,5463,2295,9069
